In [ ]:
"""Scrapper to download dog photos by age category from Petfinder.com
Note that the website is being modified and therefore some modifications can be requred."""

In [ ]:
import os
import datetime
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from urllib import request
from pathlib import Path

In [ ]:
DRIVER_PATH = Path(r"web_scrapper\chromedriver.exe")

In [ ]:
driver = webdriver.Chrome(DRIVER_PATH)

In [ ]:
driver.get('https://www.petfinder.com/')

In [ ]:
driver.find_element_by_css_selector('.findAPetMenu-button_dogs').click()

In [ ]:
## selecting searched aera (Anywhere for loading whole Petfinder datset)
select_element = driver.find_element(by = By.TAG_NAME,value = 'select')
select_object = Select(select_element)
select_object.select_by_value('Anywhere')
driver.find_element(by=By.TAG_NAME, value = 'button').click()

In [ ]:
age1 =  'Adult'
age2 = 'Senior'
age3 = 'Young'
loc = f'//div[contains(text(),"{age1}")]'

In [ ]:
## selecting dog's age category. To change from young please change the name in finding element by XPATH to Adult or Senior
driver.find_element(by=By.CSS_SELECTOR, value = '#age-select_List_Box_Btn').click()
time.sleep(7)
driver.find_element(by=By.XPATH, value = f'//div[contains(text(),"Young")]').click()
time.sleep(1)
driver.find_element(by=By.CSS_SELECTOR, value = '#age-select_List_Box_Btn').click()


In [ ]:
CATEGORIES = ['Puppy','Young','Adult', 'Senior']
# 'Puppy','Young','Adult',
PATH = str(r'A:\Downloaded_imgs') ##specify where you want to save the files


In [ ]:
def calculate_end_page():
    item = driver.find_element(by=By.XPATH, value = f"//div[@id='page-select_List_Box_Btn']")
    text = (item.text)
    text = text.removeprefix('PAGE ')
    num, denum =  text.split('/')
    num = int(num)
    denum = int(denum)
    check_val = num/denum
    print (check_val)
    return check_val

In [ ]:
for category in CATEGORIES:
    filepath = os.path.join(PATH,category)
    check = True
    driver = webdriver.Chrome(DRIVER_PATH)
    driver.get('https://www.petfinder.com/')
    time.sleep(5)
    driver.find_element_by_css_selector('.findAPetMenu-button_dogs').click()
    time.sleep(5)
    ## selecting searched aera (Anywhere for loading whole Petfinder datset)
    select_element = driver.find_element(by = By.TAG_NAME,value = 'select')
    select_object = Select(select_element)
    select_object.select_by_value('Anywhere')
    driver.find_element(by=By.TAG_NAME, value = 'button').click()
    ## selecting dog's age category
    time.sleep(5)
    driver.execute_script("window.scroll(0, 0);")
    driver.find_element(by=By.CSS_SELECTOR, value = '#age-select_List_Box_Btn').click()
    time.sleep(5)
    driver.find_element(by=By.XPATH, value = f'//div[contains(text(),"{category}")]').click()
    time.sleep(3)
    driver.find_element(by=By.CSS_SELECTOR, value = '#age-select_List_Box_Btn').click()
    all_elements = []
    ## slowly scroll to the end of page - let the imgs load
    while True:
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for i in range(1, total_height, 5):
            driver.execute_script("window.scrollTo(0, {});".format(i))
        ## get photos from the page and urls
        temp_elements = []
        page_elements = driver.find_elements(by=By.CLASS_NAME, value = 'petCard-media-img')
        for imgs in page_elements:
            url = (imgs.get_attribute('src'))
            all_elements.append(url)
            temp_elements.append(url)
            #else:
        ## download the urls
        for url in temp_elements:
            date_name = datetime.datetime.now().strftime("%y%m%d_%H%M%S%f")
            temp_name = os.path.join(filepath,date_name)
            temp_name = temp_name + str('.png')
            try:
                request.urlretrieve(url,temp_name)
            except:
                print("Cannot download image from URL. Unsupported media type")
        item = driver.find_element(by=By.XPATH, value = f"//div[@id='page-select_List_Box_Btn']")
        text = (item.text)
        print(text)
        text = text.removeprefix('PAGE ')
        num, denum =  text.split('/')
        num = int(num)
        denum = int(denum)
        check_val = num/denum
        print (check_val)
        temp_elements = []
        if check_val >= 1.0:
            print(f"Reached last page for {category} category. Closing browser.")
            driver.close()
            break
        else:
            driver.find_element(by=By.XPATH, value = '//body/pf-app[1]/div[4]/div[1]/div[1]/div[2]/pfdc-animal-search[1]/div[1]/div[1]/div[2]/div[2]/div[2]/pfdc-element[2]/div[1]/div[1]/div[2]/pfdc-page-controls[1]/div[1]/div[3]/button[1]').click()


In [ ]:
from collections import Counter
src = r"A:\Downloaded_imgs" #path to downloaded photos

def file_hash(filepath):
    """Read hash of given file"""
    with open(filepath, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

#iterate over downloaded photos and remove reocurrences
for folder in os.listdir(src):
    folderpath = os.path.join(src,folder)
    hash_list = []
    for filename in os.listdir(folderpath):
        
        temp_path = os.path.join(src,filename)
        temp_hash = file_hash(temp_path)
        if (hash_list.count(temp_hash) != 0):
            print("Reocurrence! Removing!")
        else:
            hash_list.append(temp_hash)
    print(len(Counter(hash_list)))